# Fetching Timber Output Data from a Web Tool

In this notebook, we will scrape some data from the [Timber Products Output (TPO) Reporting Tool](https://www.fs.usda.gov/srsfia/php/tpo_2009/tpo_rpa_int1.php) produced by the US Forest Service. 

The TPO reporting tool contains two primary data sources: TPO and RPA.

* **TPO:** Most US States collect data for state-level Timber Products Output (TPO) reports every 2-5 years. The years available are unique for each state, but are often collected and released more frequent than the regional or national reports.

* **RPA:** Since 1997, regional and national reports are produced covering all states in a region or across the USA every five years. This reporting is mandated by the Resource Protection Act (RPA), and these reports are known as RPA reports. The TPO Reporting Tool contains RPA data up until 2012 (2017 data are not yet integrated into the tool).

In [1]:
import requests
import pandas as pd
import numpy as np
import csv

We will use the `requests` Python library to start a `session` to keep track of things like cookies and headers that will be sent and received to/from the TPO Reporting Tool.

This example focuses on extracting data from a custom table generated by the TPO Reporting Tool that display the output of industrial roundwood by each owner type (e.g., State & Local, Federal, Private) in each county. Each table will includes data for one state from one year. It is definitely possible to generalize and modify this code to query multiple states and regions or request other data tables, but we will focus here on a more limited use case of just scraping the data we need. 

## Get a FIPS lookup table
The TPO reporting tool identifies states using Federal Information Processing Standard (FIPS) codes. We'll grab a list of these codes and create a lookup table to help identify which states we're working with and allow requests to be made using the 2-letter alphabetical codes for a state (e.g., AL, OR, TX, etc.).

In [2]:
FIPS_URL ='https://en.wikipedia.org/wiki/Federal_Information_Processing_Standard_state_code'
state_fips = pd.read_html(FIPS_URL, header=0)[0].rename({
    'Name': 'STATE_NAME', 'Alpha code': 'ALPHA_CODE', 'Numeric code': 'FIPS'
}, axis=1).dropna()
state_fips = state_fips.loc[state_fips.Status.str.contains('State;')].drop('Status', axis=1)
state_fips['FIPS'] = state_fips['FIPS'].apply(lambda x: '{:02d}'.format(x))
lookup_fips_by_state = state_fips.set_index('ALPHA_CODE')
lookup_state_by_fips = state_fips.set_index('FIPS')
lookup_fips_by_state.head()

,STATE_NAME,FIPS
ALPHA_CODE,,
AL,Alabama,01
AK,Alaska,02
AZ,Arizona,04
AR,Arkansas,05
CA,California,06


## Get a TPO lookup table
TPO reports are released in different years for each state and based on different data-collection years as well. We need to identify which report year and which data year in our requests to the TPO Reporting Tool. To help us with this, I've scraped all the different report-year and data-year pairs for each State as a text file we'll use here.

In [3]:
# this is a text file that contains all years for which TPO data are available for each state
tpo_reports = pd.read_csv('C:/GitHub/embodied_carbon/src/tpo_data.csv')

# we format it a bit to help with queries
tpo_reports['FIPS'] = tpo_reports['FIPS'].apply(lambda x: '{:02d}'.format(x))
tpo_reports['REPORT_YEAR'] = tpo_reports['REPORT_YEAR'].apply(lambda x: None if x == 'None' else int(x)).astype('Int64')
tpo_reports['DATA_YEAR'] = tpo_reports['DATA_YEAR'].apply(lambda x: None if x == 'None' else int(x)).astype('Int64')
# some states don't have any TPO reports, let's drop them
tpo_reports = tpo_reports.dropna()
tpo_reports['DATA_YEAR'] = tpo_reports['DATA_YEAR'].astype(str)

# some states also have the same year's TPO data in multiple subsequent annual reports
# here we will drop those rows which have the same data_year as an earlier year
tpo_reports = tpo_reports.drop_duplicates(subset=['STATE', 'DATA_YEAR'], keep='last').reset_index(drop=True)
tpo_reports = tpo_reports.set_index(['STATE', 'DATA_YEAR'])
tpo_reports.head()

FIPS  REPORT_YEAR
STATE DATA_YEAR                  
AL    2015        01         2015
      2013        01         2013
      2011        01         2011
      2009        01         2009
      2007        01         2007

## Define data-fetching functions for TPO and RPA data
Interacting with the TPO Reporting Tool generally follows three steps:  

1. **Define data source as TPO or RPA**  
Both reports contain comparable data and provide the same output table formats.  

2. **Define the US States and years to retrieve data**  
One or more years and one or more States can be requested  

3. **Select the table to view**  
We will retrieve the table and read it into a pandas DataFrame  

We will use three functions to extract TPO and RPA data. The:
* `fetch_rpa_by_county` will retrieve RPA data as an html table for a single state and year.
* `fetch_tpo_by_county` will retrieve TPO data as an html table for a single state and year.
* `parse_custom_table_html` parses the html table returned by the two fetching functions, and returns a nicely-formatted Pandas DataFrame.

In [4]:
def fetch_rpa_by_county(state_alpha_code, year):
    # We will GET the landing page so that we can receive a SESSION ID cookie
    START_URL = 'https://www.fs.usda.gov/srsfia/php/tpo_2009/tpo_rpa_int1.php'
    # We will POST to the next URL to define the type of data requested (TPO or RPA)
    STEP1_URL = 'https://www.fs.usda.gov/srsfia/php/tpo_2009/tpo_rpa_int2.php'
    # We will POST to the next URL to define the US State and Year(s) for which we want data
    STEP2_URL = 'https://www.fs.usda.gov/srsfia/php/tpo_2009/tpo_rpa_int4_public.php' # POST states and years
    # We will GET the next URL to retrieve a table that 
    TABLE_URL = 'https://www.fs.usda.gov/srsfia/php/tpo_2009/tpo_table_custom_vol_roundwood_by_product_int.php' # GET table output with query params

    PAYLOAD_1 = {
        'tpo_type': 'NAT',
        'submit1': 'Continue'
    }
    
    PAYLOAD_2 = {
        'units': '1000',
        'show_query': '0',
        'row1': 'CO',
        'col1': 'OWN',
        'tpo_or_rpa': 'RPA',
        'submit1': 'Continue',
    }    
    
    QUERY_PARAMS = {
        'rpatpo': 'FS_SRS_FIA_RPA_TPO.rpa_tpo_1997_2002_2007_state',
        'state_abbr_include': None,
        'tpo_or_rpa': 'RPA',
        'units': 1000,
        'row1': 'CO',
        'col1': 'OWN',
        'show_query': 0,
    }
    
    # lookup the fips code for this state
    fips_code = lookup_fips_by_state.loc[state_alpha_code]['FIPS']
    
    # form fields filled out for the state and year we want
    update_payload_2 = {
        f'Sta_{fips_code}': 'All',
        f'Sty_{fips_code}': year,
        'yr1': year,
    }
    payload_2 = dict(PAYLOAD_2, **update_payload_2)    
    
    update_query_params = {
        'state_abbr_include': state_alpha_code,
        'yr1': year,
    }
    query_params = dict(QUERY_PARAMS, **update_query_params)
    
    with requests.session() as s:
        s.get(START_URL)
        s.post(STEP1_URL, data=PAYLOAD_1)
        s.post(STEP2_URL, data=payload_2)
        response = s.get(TABLE_URL, params=query_params)
    
    df = parse_custom_table_html(response.content, state_alpha_code, year)
    return df

In [5]:
def fetch_tpo_by_county(state_alpha_code, data_year):
    # We will GET the landing page so that we can receive a SESSION ID cookie
    START_URL = 'https://www.fs.usda.gov/srsfia/php/tpo_2009/tpo_rpa_int1.php'
    # We will POST to the next URL to define the type of data requested (TPO or RPA)
    STEP1_URL = 'https://www.fs.usda.gov/srsfia/php/tpo_2009/tpo_rpa_int2.php'
    # We will POST to the next URL to define the US State and Year(s) for which we want data
    STEP2_URL = 'https://www.fs.usda.gov/srsfia/php/tpo_2009/tpo_rpa_int4_public.php' # POST states and years
    # We will GET the next URL to retrieve a table that 
    TABLE_URL = 'https://www.fs.usda.gov/srsfia/php/tpo_2009/tpo_table_custom_vol_roundwood_by_product_int.php' # GET table output with query params

    PAYLOAD_1 = {
        'tpo_type': 'TPO',
        'submit1': 'Continue'
    }
    
    PAYLOAD_2 = {
        'units': '1000',
        'show_query': '0',
        'row1': 'CO',
        'col1': 'OWN',
        'tpo_or_rpa': 'TPO',
        'submit1': 'Continue',
    }    
    
    QUERY_PARAMS = {
        'rpatpo': 'FS_SRS_FIA_RPA_TPO.srstpo_state',
        'tpo_or_rpa': 'TPO',
        'units': 1000,
        'row1': 'CO',
        'col1': 'OWN',
        'show_query': 0,
    }

    # lookup the available data years for this state
    try:
        fips_code, rept_yr = tpo_reports.loc[state_alpha_code, str(data_year)]
    except KeyError:
        avail_values = np.sort(tpo_reports.loc[state_alpha_code].
                               index.get_level_values(-1).values.astype(int))
        msg = ' '.join([f'{data_year} not available for {state_alpha_code}.',
                       f'Try one of {avail_values}.'])
        raise KeyError(msg)
    
    # form fields filled out for the state and year we want
    update_payload_2 = {
        f'Sta_{fips_code}': 'All',
        f'Sty_{fips_code}': f'{rept_yr} Report year ({data_year} data)',
        'yr1': rept_yr,
    }
    payload_2 = dict(PAYLOAD_2, **update_payload_2)    
    
    update_query_params = {
        'state_abbr_include': state_alpha_code,
        'yr1': rept_yr,
        'yr1_hdr': rept_yr,
    }
    query_params = dict(QUERY_PARAMS, **update_query_params)
    
    with requests.session() as s:
        s.get(START_URL)
        s.post(STEP1_URL, data=PAYLOAD_1)
        s.post(STEP2_URL, data=payload_2)
        response = s.get(TABLE_URL, params=query_params)
    
    df = parse_custom_table_html(response.content, state_alpha_code, data_year,
                                addl_fields={'REPORT_YEAR': rept_yr})
    
    return df

In [12]:
def parse_custom_table_html(html, state_alpha_code, year, addl_fields=None):
    df = pd.read_html(html, 
                      skiprows = [0,2], 
                      header=0
                     )[0].iloc[:-2]
    df.insert(0, 'STATE', state_alpha_code)
    df.insert(1, 'STATE_FIPS', 
              df['ST/CO'].apply(lambda x: '{:02d}'.format(int(x.split(',')[0]))))
    df.insert(2, 'YEAR', year)
    df.insert(3, 'COUNTY', 
              df['ST/CO'].apply(lambda x: x.split(',')[-1]))
    if addl_fields is not None:
        i = 4
        for key, value in addl_fields.items():
            df.insert(i, key, value)
            i+=1
            
    DATA_COLS = [
        'OWN', 'SW_10 Sawlogs', 'HW_10 Sawlogs', 'SW_20 Veneer', 'HW_20 Veneer', 
        'SW_30 Pulpwood', 'HW_30 Pulpwood', 'SW_40 Composite', 'HW_40 Composite',
        'SW_50 Fuelwood', 'HW_50 Fuelwood', 'SW_60 Posts Poles Pilings',
        'HW_60 Posts Poles Pilings', 'SW_90 Other', 'HW_90 Other',
        'SW All Products', 'HW All Products', 'All Products',
    ]
    df[DATA_COLS] = df[DATA_COLS].astype(int)
    df = df.drop(['ST/CO'], axis=1)
    return df

# Scraping data for every US State
Now, we'll scrape the TPO and RPA tables for every US State and year we can get our hands on.

In [8]:
rpa_dfs = []
for state in pd.unique(lookup_fips_by_state.index):
    state_name = lookup_fips_by_state.loc[state]['STATE_NAME']
    for year in [1997,2002,2007,2012]:
        try:
            rpa_df = fetch_rpa_by_county(state, year)
            rpa_dfs.append(rpa_df)
        except:
            print(state, year)
    print(f'Done with {state_name}.')

Done with Alabama.
Done with Alaska.
Done with Arizona.
Done with Arkansas.
Done with California.
Done with Colorado.
Done with Connecticut.
Done with Delaware.
Done with Florida.
Done with Georgia.
Done with Hawaii.
Done with Idaho.
Done with Illinois.
Done with Indiana.
Done with Iowa.
Done with Kansas.
Done with Kentucky.
Done with Louisiana.
Done with Maine.
Done with Maryland.
Done with Massachusetts.
Done with Michigan.
Done with Minnesota.
Done with Mississippi.
Done with Missouri.
Done with Montana.
Done with Nebraska.
Done with Nevada.
Done with New Hampshire.
Done with New Jersey.
Done with New Mexico.
Done with New York.
Done with North Carolina.
Done with North Dakota.
Done with Ohio.
Done with Oklahoma.
Done with Oregon.
Done with Pennsylvania.
Done with Rhode Island.
Done with South Carolina.
Done with South Dakota.
Done with Tennessee.
Done with Texas.
Done with Utah.
Done with Vermont.
Done with Virginia.
Done with Washington.
Done with West Virginia.
Done with Wisconsi

In [14]:
tpo_dfs = []
for state in pd.unique(tpo_reports.index.get_level_values(0)):
    state_name = lookup_fips_by_state.loc[state]['STATE_NAME']
    print(f'Starting {state_name}.', end=' ')
    
    num_yrs = 0
    for year in tpo_reports.loc[state].index:
        try:
            tpo_df = fetch_tpo_by_county(state, year)
            tpo_dfs.append(tpo_df)
            num_yrs += 1
        except:
            print(state, year)
            
    print(f'Done. Fetched {num_yrs} years of data.')

Starting Alabama. Done. Fetched 10 years of data.
Starting Alaska. Done. Fetched 5 years of data.
Starting Arizona. Done. Fetched 6 years of data.
Starting Arkansas. Done. Fetched 9 years of data.
Starting California. Done. Fetched 5 years of data.
Starting Colorado. Done. Fetched 5 years of data.
Starting Florida. Done. Fetched 10 years of data.
Starting Georgia. Done. Fetched 11 years of data.
Starting Idaho. Done. Fetched 5 years of data.
Starting Kentucky. Done. Fetched 11 years of data.
Starting Louisiana. Done. Fetched 9 years of data.
Starting Mississippi. Done. Fetched 9 years of data.
Starting Montana. Done. Fetched 6 years of data.
Starting Nevada. Done. Fetched 6 years of data.
Starting New Mexico. Done. Fetched 6 years of data.
Starting North Carolina. Done. Fetched 11 years of data.
Starting Oklahoma. Done. Fetched 8 years of data.
Starting Oregon. Done. Fetched 5 years of data.
Starting South Carolina. Done. Fetched 11 years of data.
Starting Tennessee. Done. Fetched 10 y

## Concatenate all the data tables, and write to disk
We'll make two large DataFrames for all the TPO data and all the RPA data, then write them to disk as text files which we'll use later on this research project. 

In [15]:
tpo_data = pd.concat(tpo_dfs, axis=0)
tpo_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19623 entries, 0 to 46
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   STATE                      19623 non-null  object
 1   STATE_FIPS                 19623 non-null  object
 2   YEAR                       19623 non-null  object
 3   COUNTY                     19623 non-null  object
 4   REPORT_YEAR                19623 non-null  int64 
 5   OWN                        19623 non-null  int32 
 6   SW_10 Sawlogs              19623 non-null  int32 
 7   HW_10 Sawlogs              19623 non-null  int32 
 8   SW_20 Veneer               19623 non-null  int32 
 9   HW_20 Veneer               19623 non-null  int32 
 10  SW_30 Pulpwood             19623 non-null  int32 
 11  HW_30 Pulpwood             19623 non-null  int32 
 12  SW_40 Composite            19623 non-null  int32 
 13  HW_40 Composite            19623 non-null  int32 
 14  SW_50 Fue

In [16]:
tpo_data.head()

,STATE,STATE_FIPS,YEAR,COUNTY,REPORT_YEAR,OWN,SW_10 Sawlogs,HW_10 Sawlogs,SW_20 Veneer,HW_20 Veneer,...,HW_40 Composite,SW_50 Fuelwood,HW_50 Fuelwood,SW_60 Posts Poles Pilings,HW_60 Posts Poles Pilings,SW_90 Other,HW_90 Other,SW All Products,HW All Products,All Products
0,AL,01,2015,Autauga,2015,2,27,2,9,0,...,0,0,0,1,0,0,0,288,100,388
1,AL,01,2015,Autauga,2015,4,1110,86,354,14,...,0,0,0,37,0,0,0,11655,4052,15706
2,AL,01,2015,Baldwin,2015,1,0,0,0,0,...,0,0,0,0,0,0,0,2,0,2
3,AL,01,2015,Baldwin,2015,2,131,6,0,14,...,0,0,0,33,0,0,0,887,105,992
4,AL,01,2015,Baldwin,2015,4,2462,117,0,272,...,0,0,0,628,0,0,0,16718,1973,18691


In [18]:
tpo_data.to_csv('../data/interim/tpo_data_by_county.csv', 
                index=False, header=True,
                quoting = csv.QUOTE_NONNUMERIC)

In [19]:
rpa_data = pd.concat(rpa_dfs, axis=0)
rpa_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15213 entries, 0 to 34
Data columns (total 22 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   STATE                      15213 non-null  object
 1   STATE_FIPS                 15213 non-null  object
 2   YEAR                       15213 non-null  int64 
 3   COUNTY                     15213 non-null  object
 4   OWN                        15213 non-null  int32 
 5   SW_10 Sawlogs              15213 non-null  int32 
 6   HW_10 Sawlogs              15213 non-null  int32 
 7   SW_20 Veneer               15213 non-null  int32 
 8   HW_20 Veneer               15213 non-null  int32 
 9   SW_30 Pulpwood             15213 non-null  int32 
 10  HW_30 Pulpwood             15213 non-null  int32 
 11  SW_40 Composite            15213 non-null  int32 
 12  HW_40 Composite            15213 non-null  int32 
 13  SW_50 Fuelwood             15213 non-null  int32 
 14  HW_50 Fue

In [21]:
rpa_data.to_csv('../data/interim/rpa_data_by_county.csv', 
                index=False, header=True,
                quoting = csv.QUOTE_NONNUMERIC)